<a href="https://colab.research.google.com/github/EdoardoMorucci/Plant-Leaves-Search-Engine---MIRCV/blob/main/retrieval_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
! pip install import-ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=8b410a0cdeaf4a8df67e7fc4b0356e836cff4c7e1a8a5859a50d48828ccdb916
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd "/content/gdrive/Shareddrives/MIRCV-PlantLeavesSearchEngine/jupyter-notebooks"

/content/gdrive/Shareddrives/MIRCV-PlantLeavesSearchEngine/jupyter-notebooks


In [5]:
import import_ipynb
import time
from statistics import median
import numpy as np
import pandas as pd
import random
import math
import pickle
from IPython.display import display
from ipywidgets import HBox, Image, HTML
from vpt_index import VantagePointTree as vpt
from vpt_index import Object as Object
from vpt_index import Node as Node


In [6]:
%cd "/content/"

/content


# Download Dataset

In [ ]:
# Dataset download 

! pip install -q kaggle

from google.colab import files
_ = files.upload()

! mkdir -p ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

! kaggle datasets download -d davidedemarco/healthy-unhealthy-plants-dataset-segmented --unzip

# downaload the noise images from https://drive.google.com/file/d/1wsrC7u104hsAU6Wo0cII33TLC3oLpTkm/view?usp=sharing


Saving kaggle.json to kaggle.json
 99% 638M/642M [00:04<00:00, 109MB/s] 
100% 642M/642M [00:04<00:00, 139MB/s]


In [ ]:
! gdown --id 1wsrC7u104hsAU6Wo0cII33TLC3oLpTkm -O noise.zip

! unzip /content/noise.zip

# Useful Paths



In [ ]:
FEATURES_PATH = "/content/gdrive/Shareddrives/MIRCV-PlantLeavesSearchEngine/total_features_new.npy"
# FEATURES_FINETUNED_PATH = "/content/gdrive/Shareddrives/MIRCV-PlantLeavesSearchEngine/final_features.npy"
DATAFRAME_PATH = "/content/gdrive/Shareddrives/MIRCV-PlantLeavesSearchEngine/total_dataframe.csv"


# VPT Index Creation

Create 

In [ ]:
features = np.load(FEATURES_PATH)
features_and_ids = []
for i in range(features.shape[0]):
  features_and_ids.append(Object(features[i],str(i)))
features.shape
vpt_index = VantagePointTree(features_and_ids, distance_measure="euclidian", size=len(features_and_ids))

In [ ]:

# query TEST

query = Object(features[72000], "Query")

start = time.time()

kNN, dNN, distance_computations = vpt_index.search_kNN(query,10)

tempo_messo = time.time() - start

print(f"Tempo impiegato {tempo_messo}")

k=10

for i in range(k):
  print(kNN[i].id)
  print(dNN[i])

print(f"Ho computato {distance_computations} su {vpt_index.size} oggetti nell'indice")

Tempo impiegato 1.6086318492889404
72000
0.0
52093
21.018135
55972
21.899681
67993
22.037794
57899
22.829988
63416
22.829988
48776
22.915493
53786
22.915493
71125
23.012262
52383
23.351599
Ho computato 39927 su 97034 oggetti nell'indice


In [ ]:
dict_tree = Node.to_dict(vpt_index.root)

with open('saved_dictionary.pkl', 'wb') as f:
    pickle.dump(dict_tree, f)

In [ ]:
def show_results(image_paths, scores, captions=None):
  """
  Shows results of a similarity search. Lists of image paths, scores, and
  (optionally) captions must be of the same length and follow the same order.
  
  Args:
    image_paths (ndarray or list): ordererd paths of images to be shown.
    scores (ndarray or list): ordered scores values to be shown next to images.
    captions (ndarray or list): ordered strings of captions to be shown; if
                                None, no caption is shown. Defaults to None.
  
  """
  if captions is None:
    captions = [''] * len(scores)  # just all empty strings

  for image_path, score, caption in zip(image_paths, scores, captions):
    title = 'score: {:.6f}'.format(score)
    display_image(image_path, title, caption)

In [ ]:
def display_image(image_path, title='', txt=''):
  """ Displays an image with optional title and text next to it using iPython
  widgets (works in Jupyter/Colab environments).
      
  Args:
    image_path (string): path to the image to show.
    title (string): header text to show next to the image.
    txt (string): text to show under the title header.
  """
  image_widget = Image.from_file(image_path, width=200, height=100)
  txt = txt.replace('\n', '<br>')
  text_widget = HTML('<p><b>{}</b></p><p>{}</p>'.format(title, txt))
  widget = HBox([image_widget, text_widget])
  display(widget)

In [ ]:
dataframe = pd.read_csv("/content/gdrive/Shareddrives/MIRCV-PlantLeavesSearchEngine/total_dataframe.csv")
dataframe

,Unnamed: 0,Image,Plant,Path
0,0,00348.jpg,Apple,/content/Healthy-Unhealthy-Plants-Dataset-Segm...
1,1,f415bc3e-3e71-4636-a3dd-78b65002384d___JR_FrgE...,Apple,/content/Healthy-Unhealthy-Plants-Dataset-Segm...
2,2,22717.jpg,Apple,/content/Healthy-Unhealthy-Plants-Dataset-Segm...
3,3,4dc8e2a8-4374-488a-afc6-fd0ea7f15c88___FREC_C....,Apple,/content/Healthy-Unhealthy-Plants-Dataset-Segm...
4,4,4b186022-c0ab-4a26-9930-c7944a3c431e___RS_HL 7...,Apple,/content/Healthy-Unhealthy-Plants-Dataset-Segm...
...,...,...,...,...
97029,97029,im8266.jpg,noise,/content/mirflickr25k/mirflickr/im8266.jpg
97030,97030,im19378.jpg,noise,/content/mirflickr25k/mirflickr/im19378.jpg
97031,97031,im14944.jpg,noise,/content/mirflickr25k/mirflickr/im14944.jpg
97032,97032,im7647.jpg,noise,/content/mirflickr25k/mirflickr/im7647.jpg


In [ ]:
paths = dataframe['Path'].to_numpy()
labels = dataframe['Plant']

kNN_paths = [paths[int(obj.id)] for obj in kNN]
kNN_labels = [labels[int(obj.id)] for obj in kNN]

show_results(kNN_paths, dNN, kNN_labels)